In [ ]:
!pip install pandas openpyxl streamlit langchain transformers faiss-cpu
!pip install huggingface_hub

!pip install langchain-huggingface

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 78.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 990.6/990.6 kB 61.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 15.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 20.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 379.9/379.9 kB 34.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.1/140.1 kB 15.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 68.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.0/83.0 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.1/141.1 kB 15.9 MB/s eta 0:00:00
  Attempting uninstall: tenacity
    Found existing installation: tenacity 9.0.0
    Uninstalling tenacity-9.0.0:
      Succes

In [ ]:
!pip install langchain_community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 63.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 4.5 MB/s eta 0:00:00


In [ ]:
!pip install -q streamlit

In [ ]:
%%writefile RAG_app.py
import streamlit as st
import pandas as pd
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationChain
from langchain.prompts import PromptTemplate
from huggingface_hub import login
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.docstore.document import Document
from langchain.chains import RetrievalQA
from langchain_community.llms import HuggingFaceEndpoint

# Initialize HuggingFace Endpoint
def initialize_llm():
    login("abcd")
    REPO_ID = "meta-llama/Meta-Llama-3-8B-Instruct"  # Replace with your model name
    HUGGINGFACE_TOKEN = "dcba"
    return HuggingFaceEndpoint(
        repo_id=REPO_ID,
        huggingfacehub_api_token=HUGGINGFACE_TOKEN,
        max_length=256,
        temperature=0.7
    )

# Function to process uploaded Excel files
def process_files(event_file, hotel_restaurant_file):
    # Load the event data
    event_df = pd.read_excel(event_file)

    # Load the hotel and restaurant data
    hotel_restaurant_df = pd.read_excel(hotel_restaurant_file)

    # Concatenate all columns into a single document for each event
    def concatenate_row(row):
        return " ".join([str(item) for item in row])

    # Create a list of Document objects for events
    event_documents = [Document(page_content=concatenate_row(row)) for _, row in event_df.iterrows()]

    # Create a list of Document objects for hotels and restaurants
    hotel_restaurant_documents = [Document(page_content=concatenate_row(row)) for _, row in hotel_restaurant_df.iterrows()]

    # Combine both lists of documents
    all_documents = event_documents + hotel_restaurant_documents

    return all_documents

# Function to create the vector store
def create_vector_store(documents):
    embedding = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
    return FAISS.from_documents(documents, embedding)

# Function to generate responses using FAISS for retrieval and maintain memory
def generate_response(query, qa):
    result = qa({"query": query})
    response = result["result"]
    return response

def main():
    st.title("Conversational RAG Chatbot with Hugging Face Endpoint and FAISS")
    st.write("Instructions: Upload your Excel files and ask any question about upcoming events, hotels, and restaurants. For example, 'Tell me upcoming events happening from 28 July 2024 to 30th July 2024.'")

    # File uploader
    event_file = st.file_uploader("Upload Event Data (Excel)", type=["xlsx"])
    hotel_restaurant_file = st.file_uploader("Upload Hotel & Restaurant Data (Excel)", type=["xlsx"])

    if event_file and hotel_restaurant_file:
        with st.spinner("Processing..."):
            # Process files and create vector store
            documents = process_files(event_file, hotel_restaurant_file)
            vectorstore = create_vector_store(documents)

            # Initialize the LLM and QA chain
            llm = initialize_llm()
            prompt_template = """
            You are an intelligent assistant. Given the user's query and the relevant data,
            provide a concise and informative response. Ensure to include the dates and details of the events,
            as well as information about hotels and restaurants. Make sure to suggest the right events according to the dates on which they are visiting.
            Do not fake answers, give answers from the provided data only. If you don't know the answer, just say that you don't know, don't try to make up an answer.
            Don't give unnecessary information.

            Context: {context}
            Question: {question}
            """
            llama_prompt = PromptTemplate(
                template=prompt_template, input_variables=["context", "question"]
            )

            chain_type_kwargs = {"prompt": llama_prompt}
            qa = RetrievalQA.from_chain_type(
                llm=llm,
                chain_type="stuff",
                retriever=vectorstore.as_retriever(),
                return_source_documents=False,
                chain_type_kwargs=chain_type_kwargs
            )

            # Initialize chat history
            if "chat_history" not in st.session_state:
                st.session_state.chat_history = []

            # Handle user queries
            while True:
                query = st.text_input("You: ")
                if query:
                    if query.lower() == "exit":
                        st.write("Bot: Goodbye!")
                        break

                    response = generate_response(query, qa)
                    st.session_state.chat_history.append(f"You: {query}")
                    st.session_state.chat_history.append(f"Bot: {response}")

                # Display the chat history
                for message in st.session_state.chat_history:
                    st.write(message)

if __name__ == '__main__':
    main()


In [ ]:
!npm install localtunnel

In [ ]:
!streamlit run my_app.py &>/content/logs.txt &

In [ ]:
!npx localtunnel --port 8501